<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:
import folium
from folium.plugins import MarkerCluster

# 1) Normalize/standardize column names
spacex_df.columns = spacex_df.columns.str.strip()

# Map common variants to standard names
rename_map = {}
if 'Launch Site' in spacex_df.columns: rename_map['Launch Site'] = 'LaunchSite'
if 'Launch_Site' in spacex_df.columns: rename_map['Launch_Site'] = 'LaunchSite'
if 'lat' in spacex_df.columns:         rename_map['lat'] = 'Latitude'
if 'Lat' in spacex_df.columns:         rename_map['Lat'] = 'Latitude'
if 'long' in spacex_df.columns:        rename_map['long'] = 'Longitude'
if 'Long' in spacex_df.columns:        rename_map['Long'] = 'Longitude'
spacex_df = spacex_df.rename(columns=rename_map)

# Identify required columns robustly
launch_col = 'LaunchSite' if 'LaunchSite' in spacex_df.columns else None
lat_col    = 'Latitude'   if 'Latitude'   in spacex_df.columns else None
lon_col    = 'Longitude'  if 'Longitude'  in spacex_df.columns else None

missing = [c for c in ['LaunchSite','Latitude','Longitude'] if c not in spacex_df.columns]
if missing:
    raise KeyError(f"Missing required columns after standardization: {missing}. "
                   f"Available columns: {list(spacex_df.columns)}")

# Ensure marker_color exists (green=success, red=failure)
if 'marker_color' not in spacex_df.columns:
    success_col = 'class' if 'class' in spacex_df.columns else ('Class' if 'Class' in spacex_df.columns else None)
    if success_col is None:
        raise KeyError("No 'class' or 'Class' column to derive marker_color from.")
    spacex_df['marker_color'] = spacex_df[success_col].apply(lambda x: 'green' if x == 1 else 'red')

# 2) Build or reuse the map
try:
    site_map
except NameError:
    center = [spacex_df[lat_col].mean(), spacex_df[lon_col].mean()]
    site_map = folium.Map(location=center, zoom_start=5, tiles='OpenStreetMap')

# 3) Add a marker cluster and drop markers for each launch
marker_cluster = MarkerCluster().add_to(site_map)

for _, row in spacex_df.dropna(subset=[lat_col, lon_col]).iterrows():
    popup_txt = f"{row[launch_col]}<br>Lat: {row[lat_col]:.4f}, Lon: {row[lon_col]:.4f}"
    folium.Marker(
        location=[row[lat_col], row[lon_col]],
        popup=folium.Popup(popup_txt, max_width=300),
        icon=folium.Icon(color=row.get('marker_color','blue'), icon_color='white', icon='info-sign')
    ).add_to(marker_cluster)

site_map

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [16]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [17]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
# Coordinates of the launch site (for example, Kennedy Space Center)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordinates of the closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the great-circle distance in km using your earlier function
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        coastline_lat, coastline_lon)

print(f"Distance from launch site to coastline: {distance_coastline:.2f} km")

Distance from launch site to coastline: 0.58 km


In [22]:
from folium.features import DivIcon

# Create a marker that displays the distance value on the map
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],  # coordinates of the closest coastline point
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)

# Add it to your map (replace site_map with your actual map variable if named differently)
distance_marker.add_to(site_map)

# Display the map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [23]:
# Define the coordinates connecting the launch site and the coastline
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# Create a PolyLine object
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,          # line thickness
    color='blue',      # color of the line
    opacity=0.8        # visibility
)

# Add the PolyLine to your map
site_map.add_child(lines)

# Display the updated map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [24]:
from folium.features import DivIcon

# Example coordinates (replace with your measured ones)
# Launch site
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Closest city
city_lat = 28.3922
city_lon = -80.6077

# Closest railway
rail_lat = 28.5635
rail_lon = -80.5708

# Closest highway
highway_lat = 28.5619
highway_lon = -80.5673

# Calculate distances using your Haversine function
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
distance_rail = calculate_distance(launch_site_lat, launch_site_lon, rail_lat, rail_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Add distance text markers to map
def add_distance_marker(lat, lon, label, distance):
    folium.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px;color:#2c3e50;">'
                 f'<b>{label}: {distance:.2f} KM</b></div>'
        )
    ).add_to(site_map)

# Add markers for each nearby feature
add_distance_marker(city_lat, city_lon, "City", distance_city)
add_distance_marker(rail_lat, rail_lon, "Railway", distance_rail)
add_distance_marker(highway_lat, highway_lon, "Highway", distance_highway)

# Optional: draw connection lines
folium.PolyLine([[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
                color='purple', weight=2, opacity=0.6).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [rail_lat, rail_lon]],
                color='orange', weight=2, opacity=0.6).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
                color='green', weight=2, opacity=0.6).add_to(site_map)

# Display map
site_map

In [25]:
# Draw lines from launch site to each nearby location

# Coordinates for reference
launch_coords = [launch_site_lat, launch_site_lon]
city_coords = [city_lat, city_lon]
rail_coords = [rail_lat, rail_lon]
highway_coords = [highway_lat, highway_lon]

# Draw line to closest city
folium.PolyLine(
    locations=[launch_coords, city_coords],
    color='purple',
    weight=2,
    opacity=0.8
).add_to(site_map)

# Draw line to closest railway
folium.PolyLine(
    locations=[launch_coords, rail_coords],
    color='orange',
    weight=2,
    opacity=0.8
).add_to(site_map)

# Draw line to closest highway
folium.PolyLine(
    locations=[launch_coords, highway_coords],
    color='green',
    weight=2,
    opacity=0.8
).add_to(site_map)

# Optional: if you added a coastline marker earlier
# folium.PolyLine(
#     locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
#     color='blue',
#     weight=2,
#     opacity=0.8
# ).add_to(site_map)

# Display map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
